In [12]:
import copy
import json
import math
import matplotlib.pyplot as plt
import mrob
import networkx as nx
import numpy as np
import open3d as o3d
import os
import pickle
from tqdm import tqdm

from sklearn.cluster import AgglomerativeClustering

from IPython.display import clear_output

import sys
sys.path.insert(1, '../src/')
import metrics

In [13]:
class CarlaLoader:
    def __init__(self, path, start_id, end_id):
        self.pcs = []
        self.Ts = []
        
        files = os.listdir(path)
        files.sort()
        meas_files = [name for name in files if 'measurements' in name]
        
        for filename in meas_files[start_id:end_id + 1]:
            with open(os.path.join(path, filename)) as jsonfile:
                data = json.load(jsonfile)
                location_json = data['playerMeasurements']['transform']['location']
                rotation_json = data['playerMeasurements']['transform']['rotation']

                if len(location_json) != 3:
                    print('{0} has no enough information in location part!'.format(filename))
                    break

                if len(rotation_json) != 3:
                    print('{0} has no enough information in rotation part!'.format(filename))
                    break

                t = np.array([location_json['y'], -location_json['x'], location_json['z']])
                euler_rad = np.array([0, 0, 
                                      rotation_json['yaw']]) / 180 * np.pi
                R = o3d.geometry.get_rotation_matrix_from_xyz(euler_rad)

                T = np.eye(4)
                T[:3, :3] = R
                T[:3, 3] = t
                self.Ts.append(T)

                pc_name = 'Lidar32_' + filename.split('_')[1].split('.')[0] + '.ply'
                pc = o3d.io.read_point_cloud(os.path.join(path, pc_name))
                self.pcs.append(pc)
        
        T0_inv = np.linalg.inv(self.Ts[0])
        self.Ts = [T0_inv @ T for T in self.Ts]

    def length(self):
        return len(self.pcs)
    
    def get(self, index):
        if 0 <= index < self.length():
            return copy.deepcopy(self.pcs[index]), self.Ts[index]
        else:
            raise ValueError('Index is out of the range')
            
    def get_pcs(self):
        return copy.deepcopy(self.pcs)
    
    def get_Ts(self):
        return self.Ts

In [14]:
def extract_orthogonal_subsets(pc, eps=1e-1, vis=False):
    
    # Estimate normals if they are not calculated for pc
    if not pc.has_normals():
        pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=2, max_nn=30))

    # Group normals    
    normals = np.asarray(pc.normals)
    clustering = AgglomerativeClustering(n_clusters=None, linkage="complete", distance_threshold=1e-1
                                     , compute_full_tree=True).fit(normals)
    
    if vis:
        pc_normals = o3d.geometry.PointCloud()
        pc_normals.points = o3d.utility.Vector3dVector(normals)
        o3d.visualization.draw_geometries([pc_normals])

    # Filter out small clusters
    MIN_CLUST_SIZE = 5
    N_clusters = np.unique(clustering.labels_).shape[0]
    labels = clustering.labels_
    huge_clusters = []
    cluster_means = []
    cluster_means_ind = []
    mp = []
    for i in range(N_clusters):
        ind = np.where(labels == i)
        if ind[0].shape[0] > MIN_CLUST_SIZE:
            huge_clusters.append(i)
            cluster_means.append(np.mean(np.vstack(normals)[ind], axis=0))
            cluster_means_ind.append(i)

            if vis:
                pcd = o3d.geometry.PointCloud()
                pcd.points = o3d.utility.Vector3dVector(np.vstack(normals)[ind])
                pcd.paint_uniform_color([0.5 - i / (2 * N_clusters), 0.5 + i / (2 * N_clusters), 
                                         0.5 - i / (2 * N_clusters)])
                mp.append(pcd)
    if vis:
        o3d.visualization.draw_geometries(mp)
    
    # Normalize means of every cluster
    cluster_means = np.vstack(cluster_means)
    cluster_means = cluster_means / np.linalg.norm(cluster_means, axis=1)[:, None]
    
    # Generate connectivity graph for normal clusters
    N = cluster_means.shape[0]
    adj_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(i):
            x = np.abs(np.dot(cluster_means[i], cluster_means[j]))       
            if x < eps:
                adj_matrix[i, j] = 1
                adj_matrix[j, i] = 1
                
    # Find max cliques
    D = nx.Graph(adj_matrix)
    x = nx.algorithms.clique.find_cliques(D)

    # Find cliques with the hugest coverage of points
    full_cliques_size = []
    full_cliques = []
    for clique in x:
        if len(clique) > 2:
            amount = 0
            for j in clique:
                amount += np.sum(labels == cluster_means_ind[j])
            full_cliques_size.append(amount)
            full_cliques.append(clique)
    
    if len(full_cliques) == 0:
        raise ValueError('AAA')

    max_ind = full_cliques_size.index(max(full_cliques_size))
    max_clique = full_cliques[max_ind]
    
    # Obtain orth subset and normals for those cliques
    pc_points = np.asarray(pc.points)
    orth_subset = [pc_points[np.where(labels == cluster_means_ind[i])[0]] for i in max_clique]
    pc_normals = np.asarray(pc.normals)
    orth_normals = [pc_normals[np.where(labels == cluster_means_ind[i])[0]] for i in max_clique]
    clique_normals = [cluster_means[i] for i in max_clique]
    return orth_subset, orth_normals, clique_normals

In [19]:
dataset_path = '/home/anastasiya/episode0/episode_00000/'
start_id = 550
end_id = 650
loader = CarlaLoader(dataset_path, start_id, end_id)

In [20]:
def trajectory_perturbation(Ts, cov=0.1):
    Ts_noise = []
    for T in Ts:
        T_noised = copy.deepcopy(T)
        T_noised[:3, 3] += [np.random.normal(0, cov), np.random.normal(0, cov), 
                            np.random.normal(0, cov)]
        Ts_noise.append(T_noised)
    return Ts_noise

In [21]:
def sampling_pipeline(pcs, T_gt, nr_metrics, fr_metric, map_tips=None, N_samples=20):
    cov_scaler = 10

    nrs = [[] for _ in range(len(nr_metrics))]
    fr = []
    
    for j in range(N_samples):
        for i in tqdm(range(cov_scaler)):
            T_pert = trajectory_perturbation(T_gt, cov = 0.1 * (i + 1) / len(pcs))
            T_pert = [np.linalg.inv(T_pert[0]) @ T for T in T_pert]
            pc_map = get_map(pcs, T_pert)
            fr.append(fr_metric(T_gt, T_pert))
            for metric_id, metric in enumerate(nr_metrics):
                nrs[metric_id].append(metric(copy.deepcopy(pc_map), map_tips=map_tips))

    return nrs, fr

def get_map(pcs, Ts):
    pc_map = o3d.geometry.PointCloud()
    for i, pc in enumerate(pcs):
        pc_map += copy.deepcopy(pc).transform(Ts[i])
        
    return pc_map

In [22]:
def orth_mme(pc_map, map_tips, knn_rad=1):
    map_tree = o3d.geometry.KDTreeFlann(pc_map)
    points = np.asarray(pc_map.points)

    orth_axes_stats = []
    orth_list = map_tips['orth_list']
    
    for k, chosen_points in enumerate(orth_list):
        metric = []
        plane_error = []
        for i in range(chosen_points.shape[0]):
            point = chosen_points[i]
            [_, idx, _] = map_tree.search_radius_vector_3d(point, knn_rad)
            if len(idx) > 5:
                metric.append(mme(points[idx]))
        
        avg_metric = np.median(metric)
    
        orth_axes_stats.append(avg_metric)

    return np.sum(orth_axes_stats)


def orth_mpv(pc_map, map_tips, knn_rad=1):
    map_tree = o3d.geometry.KDTreeFlann(pc_map)
    points = np.asarray(pc_map.points)

    orth_axes_stats = []
    orth_list = map_tips['orth_list']
    
    for k, chosen_points in enumerate(orth_list):
        metric = []
        plane_error = []
        for i in range(chosen_points.shape[0]):
            point = chosen_points[i]
            [_, idx, _] = map_tree.search_radius_vector_3d(point, knn_rad)
            if len(idx) > 5:
                metric.append(mpv(points[idx]))

        avg_metric = np.median(metric)
    
        orth_axes_stats.append(avg_metric)
    
    return np.sum(orth_axes_stats)

def mme(points):
    cov = np.cov(points.T)
    det = np.linalg.det(2 * np.pi * np.e * cov)
    return 0.5 * np.log(det) if det > 0 else -math.inf

def mpv(points):
    cov = np.cov(points.T)
    eigenvalues = np.linalg.eig(cov)[0]
    return min(eigenvalues)

In [23]:
def orth_lambda(pc_map, map_tips, knn_rad=1):
    map_tree = o3d.geometry.KDTreeFlann(pc_map)
    orth_list = map_tips['orth_list']
    orth_normals = map_tips['orth_normals']

    points = np.asarray(pc_map.points)
    three_axes_shift = []
    for k, chosen_points in enumerate(orth_list):
        metric = []
        plane_error = []
        cnt = 0
        ch_normals = orth_normals[k]
        for i in range(chosen_points.shape[0]):
            point = chosen_points[i]
            normal = ch_normals[i]
            [_, idx, _] = map_tree.search_radius_vector_3d(point, knn_rad)
            plane_error = []
            for idx_j in idx:
                plane_error.append(np.dot(points[idx_j] - point, normal))
            if len(plane_error) > 5:
                metric.append(np.cov(plane_error))

        avg_metric = np.sum(metric) / len(metric)

        three_axes_shift.append(avg_metric)
    return np.sum(three_axes_shift)

In [24]:
def build_normals_and_lambdas(pc, knn_rad=1):
    pc_tree = o3d.geometry.KDTreeFlann(pc)
    points = np.asarray(pc.points)
    main_normals = np.asarray(pc.normals)
    normals = []
    lambdas = []
    new_points = []
    for i in range(points.shape[0]):
        point = points[i]
        [k, idx, _] = pc_tree.search_radius_vector_3d(point, knn_rad)
        if len(idx) > 3:
            cov = np.cov(points[idx].T)
            eigenvalues, eigenvectors = np.linalg.eig(cov)
            idx = eigenvalues.argsort()
            eigenvalues = eigenvalues[idx]
            eigenvectors = eigenvectors[:, idx]
            if 3 * eigenvalues[0] < eigenvalues[1]:
                normals.append(main_normals[i])
                lambdas.append(eigenvalues[0])
                new_points.append(point)
            
    return np.vstack(normals), lambdas, np.vstack(new_points)

In [11]:
internal_id = 70

pc = loader.get(internal_id)[0]
pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1.5, max_nn=30))

normals, lambdas, new_points = build_normals_and_lambdas(pc)

cut_pcd = o3d.geometry.PointCloud()
cut_pcd.points = o3d.utility.Vector3dVector(new_points)
cut_pcd.normals = o3d.utility.Vector3dVector(normals)

orth_subset, orth_normals, cluster_normals = extract_orthogonal_subsets(cut_pcd, eps=10e-2, vis=False)

for map_size in [5, 15, 30]:
    print('Map size: ', map_size)


    pcs = copy.deepcopy(loader.get_pcs()[internal_id:internal_id + map_size])
    T_gt = copy.deepcopy(loader.get_Ts()[internal_id:internal_id + map_size])
    T_gt = [np.linalg.inv(T_gt[0]) @ T for T in T_gt]

    tips = {}
    tips['orth_list'] = orth_subset
    tips['orth_normals'] = orth_normals

    nr_metrics = [orth_mpv]
    nrs, fr = sampling_pipeline(pcs, T_gt, nr_metrics, metrics.rpe, tips, N_samples=10)
    save_dict = {}
    save_dict['nrs'] = nrs
    save_dict['fr'] = fr

    with open(str(start_id) + '-' + str(internal_id) + '-' + str(map_size) + '-density-r.pkl', 'wb') as sfile:
            pickle.dump(save_dict, sfile)

  0%|          | 0/10 [00:00<?, ?it/s]

Map size:  5


  0%|          | 0/10 [00:00<?, ?it/s]

Map size:  15


  0%|          | 0/10 [00:00<?, ?it/s]

Map size:  30


100%|██████████| 10/10 [00:07<00:00,  1.26it/s]
